In [5]:
import tensorflow as tf
import string
import requests
import re
from datasets import load_dataset
dataset = load_dataset("wiki_qa")

print(dataset['train'][0])

{'question_id': 'Q1', 'question': 'how are glacier caves formed?', 'document_title': 'Glacier cave', 'answer': 'A partly submerged glacier cave on Perito Moreno Glacier .', 'label': 0}


In [6]:
query_list = {}
for train_data in dataset['train']:
    if train_data['question_id'] not in query_list:
        query_list[train_data['question_id']] = train_data['question']
for train_data in dataset['validation']:
    if train_data['question_id'] not in query_list:
        query_list[train_data['question_id']] = train_data['question']
    
print(query_list['Q1'])
print(len(query_list.keys()))

how are glacier caves formed?
2414


In [7]:
def Clean_data(data):
    """Removes all the unnecessary patterns and cleans the data to get a good sentence"""
    repl='' #String for replacement
    
    #removing all open brackets
    data=re.sub('\(', repl, data)
    
    #removing all closed brackets
    data=re.sub('\)', repl, data)
    
    #Removing all the headings in data
    for pattern in set(re.findall("=.*=",data)):
        data=re.sub(pattern, repl, data)
    
    #Removing unknown words in data
    for pattern in set(re.findall("<unk>",data)):
        data=re.sub(pattern,repl,data)
    
    #Removing all the non-alphanumerical characters
    for pattern in set(re.findall(r"[^\w ]", data)):
        repl=''
        if pattern=='-':
            repl=' '
        #Retaining period, apostrophe
        if pattern!='.' and pattern!="\'":
            data=re.sub("\\"+pattern, repl, data)
            
    return data

In [19]:
cleaned_data = []
for q_index, question in query_list.items():
    data = Clean_data(question)
    cleaned_data.append(data)
print(cleaned_data[1])
print(len(cleaned_data))

How are the directions of the velocity and force vectors related in a circular motion
2414


In [31]:
def create_bigram_data(data):
    # Tokenize the sentence into words
    words = sen.split()

    # Generate all trigrams from the list of words
    trigram = [' '.join(words[i:i+2]) for i in range(len(words)-1)]

    return trigram

In [33]:
# sentence = 'how are glacier caves formed'
bigrams = []
for sen in cleaned_data:
    bi = create_bigram_data(sen)
    bigrams += bi
print(bigrams)

['how are', 'are glacier', 'glacier caves', 'caves formed', 'How are', 'are the', 'the directions', 'directions of', 'of the', 'the velocity', 'velocity and', 'and force', 'force vectors', 'vectors related', 'related in', 'in a', 'a circular', 'circular motion', 'how did', 'did apollo', 'apollo creed', 'creed die', 'how long', 'long is', 'is the', 'the term', 'term for', 'for federal', 'federal judges', 'how a', 'a beretta', 'beretta model', 'model 21', '21 pistols', 'pistols magazines', 'magazines works', 'how a', 'a vul', 'vul works', 'how an', 'an outdoor', 'outdoor wood', 'wood boiler', 'boiler works', 'how big', 'big did', 'did girl', 'girl scout', 'scout cookie', 'cookie boxes', 'boxes used', 'used to', 'to be', 'how big', 'big is', 'is the', 'the purdue', 'purdue greek', 'greek system', 'how big', 'big do', 'do sebaceous', 'sebaceous cysts', 'cysts get', 'how are', 'are pointe', 'pointe shoes', 'shoes made', 'how much', 'much is', 'is 1', '1 tablespoon', 'tablespoon of', 'of wat

In [25]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [54]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(bigrams)
sequences = tokenizer.texts_to_sequences(bigrams)

In [55]:
#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)
print(vocabulary_size)

4503


In [59]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

In [60]:
X[0]

array([8])

In [61]:
y[0]

6

In [62]:
vocab_size = len(tokenizer.word_index) + 1

In [63]:
y = to_categorical(y, num_classes = vocab_size)
X.shape[1]

1

In [64]:
seq_length = X.shape[1]

In [65]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(256, return_sequences = True))
model.add(LSTM(256))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

In [66]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 50)             225200    
                                                                 
 lstm (LSTM)                 (None, 1, 256)            314368    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 4504)              581016    
                                                                 
Total params: 1695304 (6.47 MB)
Trainable params: 169530

In [67]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [68]:
model.fit(X, y, batch_size = 200, epochs = 50)

Epoch 1/50


72/72 [==============================] - 8s 46ms/step - loss: 7.6266 - accuracy: 0.0658
Epoch 2/50
72/72 [==============================] - 3s 42ms/step - loss: 6.6118 - accuracy: 0.0655
Epoch 3/50
72/72 [==============================] - 3s 47ms/step - loss: 6.4815 - accuracy: 0.0660
Epoch 4/50
72/72 [==============================] - 3s 46ms/step - loss: 6.4067 - accuracy: 0.0676
Epoch 5/50
72/72 [==============================] - 3s 43ms/step - loss: 6.3323 - accuracy: 0.0729
Epoch 6/50
72/72 [==============================] - 3s 42ms/step - loss: 6.2666 - accuracy: 0.1121
Epoch 7/50
72/72 [==============================] - 3s 43ms/step - loss: 6.1359 - accuracy: 0.1284
Epoch 8/50
72/72 [==============================] - 3s 45ms/step - loss: 6.0166 - accuracy: 0.1324
Epoch 9/50
72/72 [==============================] - 3s 45ms/step - loss: 5.8829 - accuracy: 0.1454
Epoch 10/50
72/72 [==============================] - 3s 43ms/step - loss: 5.7250 - accuracy: 0.1705
Epoch 1

In [76]:
seed_text="harry"
seed_text

'harry'

In [77]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')
        
        # Use predict instead of predict_classes
        y_predict = model.predict(encoded)
        
        predicted_word_index = np.argmax(y_predict)
        predicted_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break

        seed_text += " " + predicted_word

    print(seed_text)

In [78]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 2)

1/1 [==============================] - 0s 33ms/step
harry potter cards
